# Simple diabete regression 

## Imports

In [ ]:
from sklearn.datasets import load_diabetes

from neuralnetlib.preprocessing import MinMaxScaler, StandardScaler
from neuralnetlib.activations import Linear, LeakyReLU
from neuralnetlib.layers import Input, Dense, Activation
from neuralnetlib.losses import MeanSquaredError, MeanAbsoluteError
from neuralnetlib.models import Sequential
from neuralnetlib.optimizers import Adam
from neuralnetlib.utils import train_test_split

## 1. Loading a dataset (in this case, the diabetes dataset)

In [11]:
x, y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## 2. Preprocessing

In [12]:
scaler_x = MinMaxScaler()
x_train = scaler_x.fit_transform(x_train)
x_test = scaler_x.transform(x_test)
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

## 3. Model definition

In [13]:
input_neurons = x_train.shape[1:][0]
num_hidden_layers = 2
hidden_neurons = 2
output_neurons = 1

model = Sequential()
model.add(Input(input_neurons))
model.add(Dense(hidden_neurons, weights_init='he_normal', random_state=42))
model.add(Activation(LeakyReLU()))

for _ in range(num_hidden_layers - 1):
    model.add(Dense(hidden_neurons, weights_init='he_normal', random_state=42))
    model.add(Activation(LeakyReLU()))

model.add(Dense(output_neurons, random_state=42))
model.add(Activation(Linear()))

## 4. Model compilation

In [14]:
model.compile(loss_function=MeanSquaredError(), optimizer=Adam(learning_rate=0.001))

model.summary()

Sequential(gradient_clip_threshold=5.0, enable_padding=False, padding_size=32, random_state=1733786806176717100)
-------------------------------------------------
Layer 1: Input(input_shape=(10,))
Layer 2: Dense(units=2)
Layer 3: Activation(LeakyReLU)
Layer 4: Dense(units=2)
Layer 5: Activation(LeakyReLU)
Layer 6: Dense(units=1)
Layer 7: Activation(Linear)
-------------------------------------------------
Loss function: MeanSquaredError
Optimizer: Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clip_norm=None, clip_value=None)
-------------------------------------------------



## 5. Model training

In [15]:
model.fit(x_train, y_train, epochs=100, batch_size=32, random_state=42)

[==============================] 100% Epoch 1/100 - 0.01s - loss: 1.2543        
[==============================] 100% Epoch 2/100 - 0.01s - loss: 1.2482        
[==============================] 100% Epoch 3/100 - 0.01s - loss: 1.2422        
[==============================] 100% Epoch 4/100 - 0.01s - loss: 1.2366        
[==============================] 100% Epoch 5/100 - 0.01s - loss: 1.2320        
[==============================] 100% Epoch 6/100 - 0.01s - loss: 1.2275        
[==============================] 100% Epoch 7/100 - 0.01s - loss: 1.2231        
[==============================] 100% Epoch 8/100 - 0.01s - loss: 1.2183        
[==============================] 100% Epoch 9/100 - 0.01s - loss: 1.2134        
[==============================] 100% Epoch 10/100 - 0.01s - loss: 1.2083       
[==============================] 100% Epoch 11/100 - 0.01s - loss: 1.2029       
[==============================] 100% Epoch 12/100 - 0.01s - loss: 1.1975       
[===========================

## 6. Model evaluation

In [16]:
loss, _ = model.evaluate(x_test, y_test)
print(f'Test loss: {loss}', "function=" + str(model.loss_function))

Test loss: 1.7401693358245864 function=MeanSquaredError


## 7. Model prediction

In [17]:
y_pred = model.predict(x_test)
print("MAE: ", MeanAbsoluteError()(y_test, y_pred))

MAE:  1.0799955616547592
